In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [2]:
from datasets import Dataset, DatasetDict, Image
from huggingface_hub import notebook_login, cached_download, hf_hub_url
import pandas as pd
from datasets import ClassLabel


In [3]:
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import os
os.chdir('/content/gdrive/MyDrive/ComputerVision')

While loading data, I omit instances with blank labels

In [131]:
def load_data(data_dir, usage, labels_csv):
  images = []
  labels = []

  for filename in os.listdir(os.path.join(data_dir, usage)):
    if filename.endswith('.jpg'):
      img_path = os.path.join(data_dir, usage, filename)
      labels_file = pd.read_csv(os.path.join(data_dir, labels_csv))
      label = labels_file[labels_file['filename'] == filename]['label']
      if not label.empty:
        labels.append(label)
        images.append(img_path)

  return images, labels

In [132]:
train_image, train_labels = load_data('Dataset_human', 'train', 'Training_set.csv')

In [136]:
def create_dataset(image_path, labels):
    def map_function(example):
        label = example['label'][0]
        return {'label': label, 'image': example['image']}

    dataset = Dataset.from_dict({'image': image_path, 'label': labels})
    dataset = dataset.map(map_function)

    return dataset

In [137]:
train_labels[0]=='hugging'

8301    True
Name: label, dtype: bool

In [138]:
train_dataset = create_dataset(train_image, train_labels)

Map:   0%|          | 0/12600 [00:00<?, ? examples/s]

In [139]:
train_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 12600
})

In [140]:
train_dataset[10]

{'image': 'Dataset_human/train/Image_8841.jpg', 'label': 'calling'}

In [141]:
dataset = train_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
dataset['validation'] = dataset.pop('test')

In [142]:
dataset_test = dataset['train'].train_test_split(test_size = 0.2, shuffle=True, seed=42)
dataset.update(dataset_test)

In [143]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8064
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 2520
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 2016
    })
})

In [144]:
dataset.push_to_hub("aslez123/human_dataset", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/555 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/aslez123/human_dataset/commit/4c2b17431c7c9d2d8b1c3f51eb85323e07cfa12c', commit_message='Upload dataset', commit_description='', oid='4c2b17431c7c9d2d8b1c3f51eb85323e07cfa12c', pr_url=None, pr_revision=None, pr_num=None)